# V2 Model Server (SKLearn)

In [1]:
import mlrun

### Configuration and package dependencies

In [2]:
%nuclio config kind="serving"
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'serving'
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


In [3]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle v3io sklearn

### Serving class code

In [4]:
from cloudpickle import load
import numpy as np
from typing import List

In [5]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [6]:
# nuclio: end-code

# Convert to function object

The sklearn-project generated one or more models that will be deployed in the server project `sklearn-servers`

In [7]:
models_path = 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'

### Document and save (as a template)

In [8]:
fn = mlrun.code_to_function('v2-model-server', description="generic sklearn model server",
                     categories=['serving', 'ml'],
                     labels={'author': 'yaronh', 'framework': 'sklearn'},
                     code_output='.')
fn.spec.default_class = 'ClassifierModel'
#print(fn.to_yaml())
fn.export()

> 2020-11-16 01:33:02,887 [info] function spec saved to path: function.yaml


### Configure and add model(s)

In [9]:
fn.add_model('mymodel', model_path=models_path)
#fn.verbose = True

# Test models locally (using a server emulator)

In [10]:
# create an emulator (mock server) from the function configuration)
server = fn.to_mock_server(globals())

{'routes': <mlrun.model.ObjectDict object at 0x7fabdcd26940>}
{'model_path': 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'}
> 2020-11-16 01:33:15,822 [info] model mymodel was loaded
> 2020-11-16 01:33:15,823 [info] Loaded ['mymodel']


### Test against the iris dataset 

In [11]:
from sklearn.datasets import load_iris
iris = load_iris()
x = iris['data'].tolist()

In [12]:
result = server.test("/v2/models/mymodel/infer", {"inputs": x})
result.keys()

dict_keys(['id', 'model_name', 'outputs'])

##  Deploy server

In [13]:
fn.deploy(dashboard='http://nuclio:8070')

> 2020-11-16 01:34:07,689 [info] deploy started
[nuclio] 2020-11-16 01:34:20,421 (info) Build complete
[nuclio] 2020-11-16 01:34:22,628 (info) Function deploy complete
[nuclio] 2020-11-16 01:34:22,631 done creating default-v2-model-server, function address: 172.17.0.1:32771
> 2020-11-16 01:34:22,672 [info] function deployed, address=172.17.0.1:32771


'http://172.17.0.1:32771'

##  Test server

In [14]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
fn.invoke('/v2/models/mymodel/infer', my_data)

{'id': 'b545df7f-2031-4dd9-99b7-fb5a32bf222a',
 'model_name': 'mymodel',
 'outputs': [0, 2]}